In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
def add_ad_to_car_dict(ad):
    car_url = ad
    response = requests.get(car_url)
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 100000
for ad in all_ads[100000:125000]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue
df_5 = pd.DataFrame.from_dict(car_dict, orient='index')
df_5.to_csv('Final_df_5.csv')

100000
100001
100002
100003
100004
100005
100006
100007
100008
100009
Completed 100010 of total 269381
100010
100011
100012
100013
100014
100015
100016
100017
100018
100019
Completed 100020 of total 269381
100020
100021
100022
100023
100024
100025
100026
100027
100028
100029
Completed 100030 of total 269381
100030
100031
100032
100033
100034
100035
100036
100037
100038
100039
Completed 100040 of total 269381
100040
100041
100042
100043
100044
100045
100046
100047
100048
100049
Completed 100050 of total 269381
100050
100051
100052
100053
100054
100055
100056
100057
100058
100059
Completed 100060 of total 269381
100060
100061
100062
100063
100064
100065
100066
100067
100068
100069
Completed 100070 of total 269381
100070
100071
100072
100073
100074
100075
100076
100077
100078
100079
Completed 100080 of total 269381
100080
100081
100082
100083
100084
100085
100086
100087
100088
100089
Completed 100090 of total 269381
100090
100091
100092
100093
100094
100095
100096
100097
100098
100099
Com

100798
100799
Completed 100800 of total 269381
100800
100801
100802
100803
100804
100805
100806
100807
100808
100809
Completed 100810 of total 269381
100810
100811
100812
100813
100814
100815
100816
100817
100818
100819
Completed 100820 of total 269381
100820
100821
100822
100823
100824
100825
100826
100827
100828
100829
Completed 100830 of total 269381
100830
100831
100832
100833
100834
100835
100836
100837
100838
100839
Completed 100840 of total 269381
100840
100841
100842
100843
100844
100845
100846
100847
100848
100849
Completed 100850 of total 269381
100850
100851
100852
100853
100854
100855
100856
100857
100858
100859
Completed 100860 of total 269381
100860
100861
100862
100863
100864
100865
100866
100867
100868
100869
Completed 100870 of total 269381
100870
100871
100872
100873
100874
100875
100876
100877
100878
100879
Completed 100880 of total 269381
100880
100881
100882
100883
100884
100885
100886
100887
100888
100889
Completed 100890 of total 269381
100890
100891
100892
10089

101592
101593
101594
101595
101596
101597
101598
101599
Completed 101600 of total 269381
101600
101601
101602
101603
101604
101605
101606
101607
101608
101609
Completed 101610 of total 269381
101610
101611
101612
101613
101614
101615
101616
101617
101618
101619
Completed 101620 of total 269381
101620
101621
101622
101623
101624
101625
101626
101627
101628
101629
Completed 101630 of total 269381
101630
101631
101632
101633
101634
101635
101636
101637
101638
101639
Completed 101640 of total 269381
101640
101641
101642
101643
101644
101645
101646
101647
101648
101649
Completed 101650 of total 269381
101650
101651
101652
101653
101654
101655
101656
101657
101658
101659
Completed 101660 of total 269381
101660
101661
101662
101663
101664
101665
101666
101667
101668
101669
Completed 101670 of total 269381
101670
101671
101672
101673
101674
101675
101676
101677
101678
101679
Completed 101680 of total 269381
101680
101681
101682
101683
101684
101685
101686
101687
101688
101689
Completed 101690 

Completed 102390 of total 269381
102390
102391
102392
102393
102394
102395
102396
102397
102398
102399
Completed 102400 of total 269381
102400
102401
102402
102403
102404
102405
102406
102407
102408
102409
Completed 102410 of total 269381
102410
102411
102412
102413
102414
102415
102416
102417
102418
102419
Completed 102420 of total 269381
102420
102421
102422
102423
102424
102425
102426
102427
102428
102429
Completed 102430 of total 269381
102430
102431
102432
102433
102434
102435
102436
102437
102438
102439
Completed 102440 of total 269381
102440
102441
102442
102443
102444
102445
102446
102447
102448
102449
Completed 102450 of total 269381
102450
102451
102452
102453
102454
102455
102456
102457
102458
102459
Completed 102460 of total 269381
102460
102461
102462
102463
102464
102465
102466
102467
102468
102469
Completed 102470 of total 269381
102470
102471
102472
102473
102474
102475
102476
102477
102478
102479
Completed 102480 of total 269381
102480
102481
102482
102483
102484
10248

103184
103185
103186
103187
103188
103189
Completed 103190 of total 269381
103190
103191
103192
103193
103194
103195
103196
103197
103198
103199
Completed 103200 of total 269381
103200
103201
103202
103203
103204
103205
103206
103207
103208
103209
Completed 103210 of total 269381
103210
103211
103212
103213
103214
103215
103216
103217
103218
103219
Completed 103220 of total 269381
103220
103221
103222
103223
103224
103225
103226
103227
103228
103229
Completed 103230 of total 269381
103230
103231
103232
103233
103234
103235
103236
103237
103238
103239
Completed 103240 of total 269381
103240
103241
103242
103243
103244
103245
103246
103247
103248
103249
Completed 103250 of total 269381
103250
103251
103252
103253
103254
103255
103256
103257
103258
103259
Completed 103260 of total 269381
103260
103261
103262
103263
103264
103265
103266
103267
103268
103269
Completed 103270 of total 269381
103270
103271
103272
103273
103274
103275
103276
103277
103278
103279
Completed 103280 of total 26938

...

121509
Completed 121510 of total 269381
121510
121511
121512
121513
121514
121515
121516
121517
121518
121519
Completed 121520 of total 269381
121520
121521
121522
121523
121524
121525
121526
121527
121528
121529
Completed 121530 of total 269381
121530
121531
121532
121533
121534
121535
121536
121537
121538
121539
Completed 121540 of total 269381
121540
121541
121542
121543
121544
121545
121546
121547
121548
121549
Completed 121550 of total 269381
121550
121551
121552
121553
121554
121555
121556
121557
121558
121559
Completed 121560 of total 269381
121560
121561
121562
121563
121564
121565
121566
121567
121568
121569
Completed 121570 of total 269381
121570
121571
121572
121573
121574
121575
121576
121577
121578
121579
Completed 121580 of total 269381
121580
121581
121582
121583
121584
121585
121586
121587
121588
121589
Completed 121590 of total 269381
121590
121591
121592
121593
121594
121595
121596
121597
121598
121599
Completed 121600 of total 269381
121600
121601
121602
121603
12160

122303
122304
122305
122306
122307
122308
122309
Completed 122310 of total 269381
122310
122311
122312
122313
122314
122315
122316
122317
122318
122319
Completed 122320 of total 269381
122320
122321
122322
122323
122324
122325
122326
122327
122328
122329
Completed 122330 of total 269381
122330
122331
122332
122333
122334
122335
122336
122337
122338
122339
Completed 122340 of total 269381
122340
122341
122342
122343
122344
122345
122346
122347
122348
122349
Completed 122350 of total 269381
122350
122351
122352
122353
122354
122355
122356
122357
122358
122359
Completed 122360 of total 269381
122360
122361
122362
122363
122364
122365
122366
122367
122368
122369
Completed 122370 of total 269381
122370
122371
122372
122373
122374
122375
122376
122377
122378
122379
Completed 122380 of total 269381
122380
122381
122382
122383
122384
122385
122386
122387
122388
122389
Completed 122390 of total 269381
122390
122391
122392
122393
122394
122395
122396
122397
122398
122399
Completed 122400 of tota

123101
123102
123103
123104
123105
123106
123107
123108
123109
Completed 123110 of total 269381
123110
123111
123112
123113
123114
123115
123116
123117
123118
123119
Completed 123120 of total 269381
123120
123121
123122
123123
123124
123125
123126
123127
123128
123129
Completed 123130 of total 269381
123130
123131
123132
123133
123134
123135
123136
123137
123138
123139
Completed 123140 of total 269381
123140
123141
123142
123143
123144
123145
123146
123147
123148
123149
Completed 123150 of total 269381
123150
123151
123152
123153
123154
123155
123156
123157
123158
123159
Completed 123160 of total 269381
123160
123161
123162
123163
123164
123165
123166
123167
123168
123169
Completed 123170 of total 269381
123170
123171
123172
123173
123174
123175
123176
123177
123178
123179
Completed 123180 of total 269381
123180
123181
123182
123183
123184
123185
123186
123187
123188
123189
Completed 123190 of total 269381
123190
123191
123192
123193
123194
123195
123196
123197
123198
123199
Completed 

123899
Completed 123900 of total 269381
123900
123901
123902
123903
123904
123905
123906
123907
123908
123909
Completed 123910 of total 269381
123910
123911
123912
123913
123914
123915
123916
123917
123918
123919
Completed 123920 of total 269381
123920
123921
123922
123923
123924
123925
123926
123927
123928
123929
Completed 123930 of total 269381
123930
123931
123932
123933
123934
123935
123936
123937
123938
123939
Completed 123940 of total 269381
123940
123941
123942
123943
123944
123945
123946
123947
123948
123949
Completed 123950 of total 269381
123950
123951
123952
123953
123954
123955
123956
123957
123958
123959
Completed 123960 of total 269381
123960
123961
123962
123963
123964
123965
123966
123967
123968
123969
Completed 123970 of total 269381
123970
123971
123972
123973
123974
123975
123976
123977
123978
123979
Completed 123980 of total 269381
123980
123981
123982
123983
123984
123985
123986
123987
123988
123989
Completed 123990 of total 269381
123990
123991
123992
123993
12399

124693
124694
124695
124696
124697
124698
124699
Completed 124700 of total 269381
124700
124701
124702
124703
124704
124705
124706
124707
124708
124709
Completed 124710 of total 269381
124710
124711
124712
124713
124714
124715
124716
124717
124718
124719
Completed 124720 of total 269381
124720
124721
124722
124723
124724
124725
124726
124727
124728
124729
Completed 124730 of total 269381
124730
124731
124732
124733
124734
124735
124736
124737
124738
124739
Completed 124740 of total 269381
124740
124741
124742
124743
124744
124745
124746
124747
124748
124749
Completed 124750 of total 269381
124750
124751
124752
124753
124754
124755
124756
124757
124758
124759
Completed 124760 of total 269381
124760
124761
124762
124763
124764
124765
124766
124767
124768
124769
Completed 124770 of total 269381
124770
124771
124772
124773
124774
124775
124776
124777
124778
124779
Completed 124780 of total 269381
124780
124781
124782
124783
124784
124785
124786
124787
124788
124789
Completed 124790 of tota